In [26]:
# 1. Importer les bibliothèques nécessaires
import pandas as pd
import json
import folium
from shapely.geometry import Polygon

In [ ]:
# 2. Charger les données tabulaires et géospatiales
faits = pd.read_csv('../dataset/generated/faits_matieres.csv')
dim_type_matiere = pd.read_csv('../dataset/generated/dim_type_matiere.csv')
dim_producteur = pd.read_csv('../dataset/generated/dim_producteur.csv')
dim_temps = pd.read_csv('../dataset/generated/dim_temps.csv')
dim_site_valorisation = pd.read_csv('../dataset/generated/dim_site_valorisation.csv')

geojson_path = "../dataset/collecte.json"
with open(geojson_path, "r") as f:
    geo_data = json.load(f)

features = geo_data['features']
geo_df = pd.json_normalize(features)
geo_df = geo_df[['properties.PROVINCE', 'properties.JOUR', 'geometry.type', 'geometry.coordinates']]
geo_df.head()

# 3. Fusionner les données sur le secteur
faits_geo = faits.merge(geo_df, left_on='geo', right_on='properties.PROVINCE', how='left')
# faits_geo.head(500)

# 4. Visualiser la carte interactive
m = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
for _, row in geo_df.iterrows():
    if row['geometry.type'] == 'Polygon':
        folium.Polygon(
            locations=[(lat, lon) for lon, lat in row['geometry.coordinates'][0]],
            color='blue',
            fill=True,
            fill_opacity=0.2,
            popup=row['properties.PROVINCE']
        ).add_to(m)
# Ajouter un marker pour chaque fait (si coordonnées disponibles)
for _, row in faits_geo.iterrows():
    if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
        # print(row['geo'], row['type_matiere'], row['volume_tonnes'])

        # Extraire tous les points du polygone
        coords = row['geometry.coordinates'][0]
        # Créer le polygone et calculer le centroïde
        poly = Polygon([(lon, lat) for lon, lat in coords])
        centroid = poly.centroid
        folium.Marker(
            location=[centroid.y, centroid.x],
            popup=f"{row['type_matiere']} ({row['volume_tonnes']} t) - {row['geo']}",
            icon=folium.Icon(color='red', icon='home')
        ).add_to(m)
m.save("../dataset/generated/collecte_secteurs.html")
print("Carte interactive générée avec markers : collecte_secteurs.html")


Carte interactive générée avec markers : collecte_secteurs.html
